In [4]:
import tensorflow as tf
import os

W = tf.Variable(tf.zeros([5,1]), name='weights')
b = tf.Variable(0., name="bias")

def combine_inputs(X):
    return tf.matmul(X,W) + b

def inference(X):
    return tf.sigmoid(combine_inputs(X))

def loss(X, Y):
    return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(combine_inputs(X), Y))

def read_csv(batch_size, file_name, record_defaults):
    filename_queue = tf.train.string_input_producer([os.path.join(os.getcwd(), file_name)])

    reader = tf.TextLineReader(skip_header_lines=1)
    key, value = reader.read(filename_queue)

    # decode_csv will convert a Tensor from type string (the text line) in
    # a tuple of tensor columns with the specified defaults, which also
    # sets the data type for each column
    decoded = tf.decode_csv(value, record_defaults=record_defaults)

    # batch actually reads the file and loads "batch_size" rows in a single tensor
    return tf.train.shuffle_batch(decoded,
                                  batch_size=batch_size,
                                  capacity=batch_size * 50,
                                  min_after_dequeue=batch_size)


def inputs():
    passenger_id, survived, pclass, name, sex, age, sibsp, parch, ticket, fare, cabin, embarked = \
        read_csv(100, "train.csv", [[0.0], [0.0], [0], [""], [""], [0.0], [0.0], [0.0], [""], [0.0], [""], [""]])

    # convert categorical data
    is_first_class = tf.to_float(tf.equal(pclass, [1]))
    is_second_class = tf.to_float(tf.equal(pclass, [2]))
    is_third_class = tf.to_float(tf.equal(pclass, [3]))

    gender = tf.to_float(tf.equal(sex, ["female"]))

    # Finally we pack all the features in a single matrix;
    # We then transpose to have a matrix with one example per row and one feature per column.
    print(is_first_class, is_second_class, is_third_class, gender, age)
    print(tf.pack([is_first_class, is_second_class, is_third_class, gender, age]))
    print(tf.transpose(tf.pack([is_first_class, is_second_class, is_third_class, gender, age])))
    features = tf.transpose(tf.pack([is_first_class, is_second_class, is_third_class, gender, age]))
    survived = tf.reshape(survived, [100, 1])

    return features, survived


def train(total_loss):
    learning_rate = 0.01
    return tf.train.GradientDescentOptimizer(learning_rate).minimize(total_loss)


def evaluate(sess, X, Y):

    predicted = tf.cast(inference(X) > 0.5, tf.float32)

    print(sess.run(tf.reduce_mean(tf.cast(tf.equal(predicted, Y), tf.float32))))

# Launch the graph in a session, setup boilerplate
with tf.Session() as sess:

    tf.initialize_all_variables().run()

    X, Y = inputs()

    total_loss = loss(X, Y)
    train_op = train(total_loss)

    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)

    # actual training loop
    training_steps = 1000
    for step in range(training_steps):
        sess.run([train_op])
        # for debugging and learning purposes, see how the loss gets decremented thru training steps
        if step % 10 == 0:
            print("loss: ", sess.run([total_loss]))

    evaluate(sess, X, Y)

    import time
    time.sleep(5)

    coord.request_stop()
    coord.join(threads)
    sess.close()


Tensor("ToFloat_4:0", shape=(100,), dtype=float32) Tensor("ToFloat_5:0", shape=(100,), dtype=float32) Tensor("ToFloat_6:0", shape=(100,), dtype=float32) Tensor("ToFloat_7:0", shape=(100,), dtype=float32) Tensor("shuffle_batch_1:5", shape=(100,), dtype=float32)
Tensor("pack_1:0", shape=(5, 100), dtype=float32)
Tensor("transpose_1:0", shape=(100, 5), dtype=float32)
loss:  [0.69863009]
loss:  [0.69091952]
loss:  [0.67558932]
loss:  [0.70747638]
loss:  [0.64735168]
loss:  [0.68613696]
loss:  [0.88511628]
loss:  [0.66866684]
loss:  [0.75413585]
loss:  [0.67646754]
loss:  [0.66340059]
loss:  [0.60800713]
loss:  [0.66030604]
loss:  [0.63432008]
loss:  [0.65650749]
loss:  [0.78220809]
loss:  [0.65129906]
loss:  [0.62849593]
loss:  [0.64724958]
loss:  [0.63912565]
loss:  [0.69799912]
loss:  [0.68846411]
loss:  [0.68124437]
loss:  [0.63501042]
loss:  [0.66396439]
loss:  [0.61787111]
loss:  [0.62554729]
loss:  [0.72677398]
loss:  [0.62738013]
loss:  [0.6726439]
loss:  [0.60488039]
loss:  [0.60894